# Correlating well log pairs: Complex Dynamic Time Warping with boundary constraints

## Introduction to dynamic time warping

In [2]:
# Data manipulation and analysis
import os
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
import glob
from IPython.display import Image as IPImage, display
from scipy import stats
from matplotlib.lines import Line2D
import matplotlib.cm as cm  # ADD THIS LINE
import matplotlib.colors as colors  # ADD THIS LINE
from itertools import combinations
warnings.filterwarnings('ignore')

from pyCoreRelator import (
    run_comprehensive_dtw_analysis,
    find_complete_core_paths,
    calculate_interpolated_ages,
    load_log_data,
    plot_core_data,
    plot_correlation_distribution,
    load_core_age_constraints,
    # New functions moved from notebook to package
    generate_constraint_subsets,
    run_multi_parameter_analysis,
    load_and_prepare_quality_data,
    reconstruct_raw_data_from_histogram,
    cohens_d,
    plot_quality_comparison,
    plot_t_statistics_vs_constraints,
    plot_quality_distributions
)

<hr>

### Define basic parameters

### Define core pairs

In [ ]:
# Define core names as variables for easy reference
# CORE_A = "M9907-22PC"
CORE_A = "M9907-23PC"
# CORE_A = "M9907-12PC"
# CORE_A = "RR0207-56PC" 

# CORE_B = "M9907-23PC"
CORE_B = "M9907-11PC"
# CORE_B = "RR0207-56PC" 

#### Log data paths and column name structure

In [ ]:
# Define log columns to extract
# LOG_COLUMNS = ['hiresMS', 'CT', 'Lumin']  # Choose which logs to include
LOG_COLUMNS = ['hiresMS']  # Choose which logs to include
DEPTH_COLUMN = 'SB_DEPTH_cm'

# Define directory paths
mother_dir = '/Users/larryslai/Library/CloudStorage/Dropbox/My Documents/University of Texas Austin/(Project) NWP turbidites/Cascadia_core_data/OSU_dataset/'

# Define paths for Core A
core_a_log_paths = {
    'hiresMS': f'{mother_dir}_compiled_logs/{CORE_A}/ML_filled/{CORE_A}_hiresMS_MLfilled.csv',
    'CT': f'{mother_dir}_compiled_logs/{CORE_A}/ML_filled/{CORE_A}_CT_MLfilled.csv',
    'Lumin': f'{mother_dir}_compiled_logs/{CORE_A}/ML_filled/{CORE_A}_RGB_MLfilled.csv',
    'R': f'{mother_dir}_compiled_logs/{CORE_A}/ML_filled/{CORE_A}_RGB_MLfilled.csv',
    'G': f'{mother_dir}_compiled_logs/{CORE_A}/ML_filled/{CORE_A}_RGB_MLfilled.csv',
    'B': f'{mother_dir}_compiled_logs/{CORE_A}/ML_filled/{CORE_A}_RGB_MLfilled.csv',
    'Den_gm/cc': f'{mother_dir}_compiled_logs/{CORE_A}/ML_filled/{CORE_A}_MST_MLfilled.csv'
}

# Define paths for Core B
core_b_log_paths = {
    'hiresMS': f'{mother_dir}_compiled_logs/{CORE_B}/ML_filled/{CORE_B}_hiresMS_MLfilled.csv',
    'CT': f'{mother_dir}_compiled_logs/{CORE_B}/ML_filled/{CORE_B}_CT_MLfilled.csv',
    'Lumin': f'{mother_dir}_compiled_logs/{CORE_B}/ML_filled/{CORE_B}_RGB_MLfilled.csv',
    'R': f'{mother_dir}_compiled_logs/{CORE_B}/ML_filled/{CORE_B}_RGB_MLfilled.csv',
    'G': f'{mother_dir}_compiled_logs/{CORE_B}/ML_filled/{CORE_B}_RGB_MLfilled.csv',
    'B': f'{mother_dir}_compiled_logs/{CORE_B}/ML_filled/{CORE_B}_RGB_MLfilled.csv',
    'Den_gm/cc': f'{mother_dir}_compiled_logs/{CORE_B}/ML_filled/{CORE_B}_MST_MLfilled.csv'
}

# Define column mapping for alternative column names
column_alternatives = {
    'hiresMS': ['MS'],
    'CT': ['CT_value'],
    'R': ['R', 'red', 'Red'],
    'G': ['G', 'green', 'Green'],
    'B': ['B', 'blue', 'Blue'],
    'Lumin': ['luminance', 'Luminance'],
    'Den_gm/cc': ['Density', 'density']
}

NameError: name 'CORE_A' is not defined

<hr>

### Load log data

In [ ]:
# Load data for Core A
log_a, md_a, _, _, _ = load_log_data(
    core_a_log_paths,
    log_columns=LOG_COLUMNS,
    depth_column=DEPTH_COLUMN,
    normalize=True,
    column_alternatives=column_alternatives
)

# Load data for Core B
log_b, md_b, _, _, _ = load_log_data(
    core_b_log_paths,
    log_columns=LOG_COLUMNS,
    depth_column=DEPTH_COLUMN,
    normalize=True,
    column_alternatives=column_alternatives
)

### Load picked depth boundaries

In [ ]:
%matplotlib inline

# Define paths to the CSV files
pickeddepth_a_csv = f'pickeddepth/{CORE_A}_pickeddepth.csv'
pickeddepth_b_csv = f'pickeddepth/{CORE_B}_pickeddepth.csv'

# Load picked depths and extract category 1 depths
if os.path.exists(pickeddepth_b_csv):
    picked_data_b = pd.read_csv(pickeddepth_b_csv)
    all_depths_b_cat1 = picked_data_b[picked_data_b['category'] == 1]['picked_depths_cm'].values.astype('float32')
else:
    print(f"Warning: {pickeddepth_b_csv} not found. Using empty array for all_depths_b_cat1.")
    all_depths_b_cat1 = np.array([]).astype('float32')

if os.path.exists(pickeddepth_a_csv):
    picked_data_a = pd.read_csv(pickeddepth_a_csv)
    all_depths_a_cat1 = picked_data_a[picked_data_a['category'] == 1]['picked_depths_cm'].values.astype('float32')
else:
    print(f"Warning: {pickeddepth_a_csv} not found. Using empty array for all_depths_a_cat1.")
    all_depths_a_cat1 = np.array([]).astype('float32')


### Load age data

In [ ]:
# Load age constraints for both cores
consider_adjacent_core = True

data_columns = {
    'age': 'calib502_agebp',
    'pos_error': 'calib502_2sigma_pos', 
    'neg_error': 'calib502_2sigma_neg',
    'min_depth': 'mindepth_cm',
    'max_depth': 'maxdepth_cm',
    'in_sequence': 'in_sequence',
    'core': 'core',
    'interpreted_bed': 'interpreted_bed'
}

# Configuration
age_base_path = '/Users/larryslai/Library/CloudStorage/Dropbox/My Documents/University of Texas Austin/(Project) NWP turbidites/Cascadia_core_data/Age constraints/Goldfinger2012'

# Load age constraints for both cores
age_data_a = load_core_age_constraints(CORE_A, age_base_path, consider_adjacent_core, data_columns, mute_mode=True)
age_data_b = load_core_age_constraints(CORE_B, age_base_path, consider_adjacent_core, data_columns, mute_mode=True)

uncertainty_method='MonteCarlo'   # 'MonteCarlo', 'Linear', or 'Gaussian'

### Compute estimated ages for each picked depth boundary

In [ ]:
# Calculate interpolated ages for Core A using the function
pickeddepth_ages_a = calculate_interpolated_ages(
    # Input data
    picked_depths=all_depths_a_cat1,                                     # depths to interpolate ages for
    age_constraints_depths=age_data_a['depths'],                         # age constraint depths
    age_constraints_ages=age_data_a['ages'],                             # age constraint ages
    age_constraints_pos_errors=age_data_a['pos_errors'],                 # positive errors
    age_constraints_neg_errors=age_data_a['neg_errors'],                 # negative errors
    age_constraints_in_sequence_flags=age_data_a['in_sequence_flags'],   # in-sequence flags
    age_constraint_source_core=age_data_a['core'],                       # source core for each constraint
    # Core boundaries
    top_bottom=True,                                                     # include top and bottom depths/ages
    top_depth=0.0,                                                       # top of core depth
    bottom_depth=md_a[-1],                                               # max depth of core a
    top_age=0,                                                           # default age at top of core
    top_age_pos_error=75,                                                # default positive uncertainty of top age
    top_age_neg_error=75,                                                # default negative uncertainty of top age
    # Uncertainty calculation
    uncertainty_method=uncertainty_method,                               # uncertainty calculation method: 'MonteCarlo', 'Linear', or 'Gaussian'
    n_monte_carlo=10000,                                                 # number of Monte Carlo iterations
    # Visualization and output
    show_plot=False,                                                      # display plot
    core_name=CORE_A,                                                    # core name for plot title
    export_csv=False,                                                    # export results to CSV
    mute_mode=True
)

# Calculate interpolated ages for Core B using the function
pickeddepth_ages_b = calculate_interpolated_ages(
    # Input data
    picked_depths=all_depths_b_cat1,                                     # depths to interpolate ages for
    age_constraints_depths=age_data_b['depths'],                         # age constraint depths
    age_constraints_ages=age_data_b['ages'],                             # age constraint ages
    age_constraints_pos_errors=age_data_b['pos_errors'],                 # positive errors
    age_constraints_neg_errors=age_data_b['neg_errors'],                 # negative errors
    age_constraints_in_sequence_flags=age_data_b['in_sequence_flags'],   # in-sequence flags
    age_constraint_source_core=age_data_b['core'],                       # source core for each constraint
    # Core boundaries
    top_bottom=True,                                                     # include top and bottom depths/ages
    top_depth=0.0,                                                       # top of core depth
    bottom_depth=md_b[-1],                                               # max depth of core b
    top_age=0,                                                           # default age at top of core
    top_age_pos_error=75,                                                # default positive uncertainty of top age
    top_age_neg_error=75,                                                # default negative uncertainty of top age
    # Uncertainty calculation
    uncertainty_method=uncertainty_method,                               # uncertainty calculation method: 'MonteCarlo', 'Linear', or 'Gaussian'
    n_monte_carlo=10000,                                                 # number of Monte Carlo sampling iterations
    # Visualization and output
    show_plot=False,                                                      # display plot
    core_name=CORE_B,                                                    # core name for plot title
    export_csv=False,                                                    # export results to CSV
    mute_mode=True
)

<hr>

## Compute quality metric distribution for all stituation

In [ ]:
# Cell: Multi-Parameter Distribution Analysis

test_age_constraint_removal = True  # Set to False to disable age constraint removal testing

# Run all parameter combinations and plot distribution curves together

# Define all parameter combinations to test
parameter_combinations = [
    # {'age_consideration': True, 'restricted_age_correlation': True, 'shortest_path_search': False},
    {'age_consideration': True, 'restricted_age_correlation': True, 'shortest_path_search': True},
    # {'age_consideration': True, 'restricted_age_correlation': False, 'shortest_path_search': False},
    # {'age_consideration': True, 'restricted_age_correlation': False, 'shortest_path_search': True},
    # {'age_consideration': False, 'restricted_age_correlation': False, 'shortest_path_search': False},
    {'age_consideration': False, 'restricted_age_correlation': False, 'shortest_path_search': True}
]

# Define all quality indices to process
target_quality_indices = ['corr_coef', 'norm_dtw', 'perc_diag']

# Set up output CSV filenames
if LOG_COLUMNS == ['hiresMS']:
    log_suffix = 'MSonly'
elif LOG_COLUMNS == ['hiresMS','CT', 'Lumin']:
    log_suffix = 'MSCTLumin'
else:
    log_suffix = 'unspecified'

output_csv_filenames = {}
for quality_index in target_quality_indices:
    if quality_index == 'corr_coef':
        output_csv_filenames[quality_index] = f'outputs/r-values_fit_params_{log_suffix}_{CORE_A}_{CORE_B}.csv'
    else:
        output_csv_filenames[quality_index] = f'outputs/{quality_index}_fit_params_{log_suffix}_{CORE_A}_{CORE_B}.csv'

# Execute the analysis function
run_multi_parameter_analysis(
    # Core data inputs
    log_a=log_a, 
    log_b=log_b, 
    md_a=md_a, 
    md_b=md_b,
    all_depths_a_cat1=all_depths_a_cat1,
    all_depths_b_cat1=all_depths_b_cat1,
    pickeddepth_ages_a=pickeddepth_ages_a,
    pickeddepth_ages_b=pickeddepth_ages_b,
    age_data_a=age_data_a,
    age_data_b=age_data_b,
    uncertainty_method=uncertainty_method,
    
    # Analysis parameters
    parameter_combinations=parameter_combinations,
    target_quality_indices=target_quality_indices,
    test_age_constraint_removal=test_age_constraint_removal,
    
    # Core identifiers
    core_a_name=CORE_A,
    core_b_name=CORE_B,
    
    # Output configuration
    output_csv_filenames=output_csv_filenames,
    
    # Optional parameters
    log_columns=LOG_COLUMNS
)

### Plotting: compare the quality metric to the null hypothesis

In [ ]:
# Define file names outside the function
target_quality_indices = ['corr_coef', 'norm_dtw', 'perc_diag', 'dtw_warp_eff']

# Set up log suffix based on LOG_COLUMNS (assuming these variables are defined earlier)
if LOG_COLUMNS == ['hiresMS']:
    log_suffix = 'MSonly'
elif LOG_COLUMNS == ['hiresMS','CT', 'Lumin']:
    log_suffix = 'MSCTLumin'
else:
    log_suffix = 'unspecified'

# Define master CSV filenames
master_csv_filenames = {}
for quality_index in target_quality_indices:
    if quality_index == 'corr_coef':
        master_csv_filenames[quality_index] = f'outputs/r-values_fit_params_{log_suffix}_{CORE_A}_{CORE_B}.csv'
    else:
        master_csv_filenames[quality_index] = f'outputs/{quality_index}_fit_params_{log_suffix}_{CORE_A}_{CORE_B}.csv'

# Define synthetic CSV filenames
synthetic_csv_filenames = {}
for quality_index in target_quality_indices:
    synthetic_csv_filenames[quality_index] = f'outputs/synthetic_PDFs_{log_suffix}_{quality_index}.csv'

# Define output figure filenames
output_figure_filenames = {}
for quality_index in target_quality_indices:
    if quality_index == 'corr_coef':
        output_figure_filenames[quality_index] = f'outputs/r-values_comparison_{log_suffix}_{CORE_A}_{CORE_B}.png'
    else:
        output_figure_filenames[quality_index] = f'outputs/{quality_index}_comparison_{log_suffix}_{CORE_A}_{CORE_B}.png'

# Use the function to produce EXACTLY the same result as the current code
plot_quality_comparison(
    target_quality_indices=target_quality_indices,
    master_csv_filenames=master_csv_filenames,
    synthetic_csv_filenames=synthetic_csv_filenames,
    output_figure_filenames=output_figure_filenames,
    CORE_A=CORE_A,
    CORE_B=CORE_B,
    debug=True  # Set to False for detailed output
)

<hr> 

## Loop Processing

In [ ]:
# Define all core options
CORE_A_OPTIONS = ["M9907-12PC", "M9907-23PC", "M9907-25PC", "M9907-30PC", "M9907-31PC", "RR0207-56PC"]
CORE_B_OPTIONS = ["M9907-11PC", "M9907-12PC", "M9907-23PC"]

# Loop through all valid combinations of CORE_A and CORE_B
for CORE_A in CORE_A_OPTIONS:
    for CORE_B in CORE_B_OPTIONS:
        # Skip if CORE_A and CORE_B are the same
        if CORE_A == CORE_B:
            continue
            
        print(f"\n{'='*80}")
        print(f"Processing core pair: {CORE_A} vs {CORE_B}")
        print(f"{'='*80}")
        
    # Define log columns to extract
    LOG_COLUMNS = ['hiresMS', 'CT', 'Lumin']  # Choose which logs to include
    # LOG_COLUMNS = ['hiresMS']  # Choose which logs to include
    DEPTH_COLUMN = 'SB_DEPTH_cm'

    # Define directory paths
    mother_dir = '/Users/larryslai/Library/CloudStorage/Dropbox/My Documents/University of Texas Austin/(Project) NWP turbidites/Cascadia_core_data/OSU_dataset/'

    # Define paths for Core A
    core_a_log_paths = {
        'hiresMS': f'{mother_dir}_compiled_logs/{CORE_A}/ML_filled/{CORE_A}_hiresMS_MLfilled.csv',
        'CT': f'{mother_dir}_compiled_logs/{CORE_A}/ML_filled/{CORE_A}_CT_MLfilled.csv',
        'Lumin': f'{mother_dir}_compiled_logs/{CORE_A}/ML_filled/{CORE_A}_RGB_MLfilled.csv',
        'R': f'{mother_dir}_compiled_logs/{CORE_A}/ML_filled/{CORE_A}_RGB_MLfilled.csv',
        'G': f'{mother_dir}_compiled_logs/{CORE_A}/ML_filled/{CORE_A}_RGB_MLfilled.csv',
        'B': f'{mother_dir}_compiled_logs/{CORE_A}/ML_filled/{CORE_A}_RGB_MLfilled.csv',
        'Den_gm/cc': f'{mother_dir}_compiled_logs/{CORE_A}/ML_filled/{CORE_A}_MST_MLfilled.csv'
    }

    # Define paths for Core B
    core_b_log_paths = {
        'hiresMS': f'{mother_dir}_compiled_logs/{CORE_B}/ML_filled/{CORE_B}_hiresMS_MLfilled.csv',
        'CT': f'{mother_dir}_compiled_logs/{CORE_B}/ML_filled/{CORE_B}_CT_MLfilled.csv',
        'Lumin': f'{mother_dir}_compiled_logs/{CORE_B}/ML_filled/{CORE_B}_RGB_MLfilled.csv',
        'R': f'{mother_dir}_compiled_logs/{CORE_B}/ML_filled/{CORE_B}_RGB_MLfilled.csv',
        'G': f'{mother_dir}_compiled_logs/{CORE_B}/ML_filled/{CORE_B}_RGB_MLfilled.csv',
        'B': f'{mother_dir}_compiled_logs/{CORE_B}/ML_filled/{CORE_B}_RGB_MLfilled.csv',
        'Den_gm/cc': f'{mother_dir}_compiled_logs/{CORE_B}/ML_filled/{CORE_B}_MST_MLfilled.csv'
    }

    # Define column mapping for alternative column names
    column_alternatives = {
        'hiresMS': ['MS'],
        'CT': ['CT_value'],
        'R': ['R', 'red', 'Red'],
        'G': ['G', 'green', 'Green'],
        'B': ['B', 'blue', 'Blue'],
        'Lumin': ['luminance', 'Luminance'],
        'Den_gm/cc': ['Density', 'density']
    }

        # Load data for Core A
        log_a, md_a, _, _, _ = load_log_data(
            core_a_log_paths,
            log_columns=LOG_COLUMNS,
            depth_column=DEPTH_COLUMN,
            normalize=True,
            column_alternatives=column_alternatives
        )

        # Load data for Core B
        log_b, md_b, _, _, _ = load_log_data(
            core_b_log_paths,
            log_columns=LOG_COLUMNS,
            depth_column=DEPTH_COLUMN,
            normalize=True,
            column_alternatives=column_alternatives
        )

        %matplotlib inline

        # Define paths to the CSV files
        pickeddepth_a_csv = f'pickeddepth/{CORE_A}_pickeddepth.csv'
        pickeddepth_b_csv = f'pickeddepth/{CORE_B}_pickeddepth.csv'

        # Load picked depths and extract category 1 depths
        if os.path.exists(pickeddepth_b_csv):
            picked_data_b = pd.read_csv(pickeddepth_b_csv)
            all_depths_b_cat1 = picked_data_b[picked_data_b['category'] == 1]['picked_depths_cm'].values.astype('float32')
        else:
            print(f"Warning: {pickeddepth_b_csv} not found. Using empty array for all_depths_b_cat1.")
            all_depths_b_cat1 = np.array([]).astype('float32')

        if os.path.exists(pickeddepth_a_csv):
            picked_data_a = pd.read_csv(pickeddepth_a_csv)
            all_depths_a_cat1 = picked_data_a[picked_data_a['category'] == 1]['picked_depths_cm'].values.astype('float32')
        else:
            print(f"Warning: {pickeddepth_a_csv} not found. Using empty array for all_depths_a_cat1.")
            all_depths_a_cat1 = np.array([]).astype('float32')

        # Load age constraints for both cores
        consider_adjacent_core = True

        data_columns = {
            'age': 'calib502_agebp',
            'pos_error': 'calib502_2sigma_pos', 
            'neg_error': 'calib502_2sigma_neg',
            'min_depth': 'mindepth_cm',
            'max_depth': 'maxdepth_cm',
            'in_sequence': 'in_sequence',
            'core': 'core',
            'interpreted_bed': 'interpreted_bed'
        }

        # Configuration
        age_base_path = '/Users/larryslai/Library/CloudStorage/Dropbox/My Documents/University of Texas Austin/(Project) NWP turbidites/Cascadia_core_data/Age constraints/Goldfinger2012'

        # Load age constraints for both cores
        age_data_a = load_core_age_constraints(CORE_A, age_base_path, consider_adjacent_core, data_columns, mute_mode=True)
        age_data_b = load_core_age_constraints(CORE_B, age_base_path, consider_adjacent_core, data_columns, mute_mode=True)

        uncertainty_method='MonteCarlo'   # 'MonteCarlo', 'Linear', or 'Gaussian'

        # Calculate interpolated ages for Core A using the function
        pickeddepth_ages_a = calculate_interpolated_ages(
            # Input data
            picked_depths=all_depths_a_cat1,                                     # depths to interpolate ages for
            age_constraints_depths=age_data_a['depths'],                         # age constraint depths
            age_constraints_ages=age_data_a['ages'],                             # age constraint ages
            age_constraints_pos_errors=age_data_a['pos_errors'],                 # positive errors
            age_constraints_neg_errors=age_data_a['neg_errors'],                 # negative errors
            age_constraints_in_sequence_flags=age_data_a['in_sequence_flags'],   # in-sequence flags
            age_constraint_source_core=age_data_a['core'],                       # source core for each constraint
            # Core boundaries
            top_bottom=True,                                                     # include top and bottom depths/ages
            top_depth=0.0,                                                       # top of core depth
            bottom_depth=md_a[-1],                                               # max depth of core a
            top_age=0,                                                           # default age at top of core
            top_age_pos_error=75,                                                # default positive uncertainty of top age
            top_age_neg_error=75,                                                # default negative uncertainty of top age
            # Uncertainty calculation
            uncertainty_method=uncertainty_method,                               # uncertainty calculation method: 'MonteCarlo', 'Linear', or 'Gaussian'
            n_monte_carlo=10000,                                                 # number of Monte Carlo iterations
            # Visualization and output
            show_plot=False,                                                      # display plot
            core_name=CORE_A,                                                    # core name for plot title
            export_csv=False,                                                    # export results to CSV
            mute_mode=True
        )

        # Calculate interpolated ages for Core B using the function
        pickeddepth_ages_b = calculate_interpolated_ages(
            # Input data
            picked_depths=all_depths_b_cat1,                                     # depths to interpolate ages for
            age_constraints_depths=age_data_b['depths'],                         # age constraint depths
            age_constraints_ages=age_data_b['ages'],                             # age constraint ages
            age_constraints_pos_errors=age_data_b['pos_errors'],                 # positive errors
            age_constraints_neg_errors=age_data_b['neg_errors'],                 # negative errors
            age_constraints_in_sequence_flags=age_data_b['in_sequence_flags'],   # in-sequence flags
            age_constraint_source_core=age_data_b['core'],                       # source core for each constraint
            # Core boundaries
            top_bottom=True,                                                     # include top and bottom depths/ages
            top_depth=0.0,                                                       # top of core depth
            bottom_depth=md_b[-1],                                               # max depth of core b
            top_age=0,                                                           # default age at top of core
            top_age_pos_error=75,                                                # default positive uncertainty of top age
            top_age_neg_error=75,                                                # default negative uncertainty of top age
            # Uncertainty calculation
            uncertainty_method=uncertainty_method,                               # uncertainty calculation method: 'MonteCarlo', 'Linear', or 'Gaussian'
            n_monte_carlo=10000,                                                 # number of Monte Carlo sampling iterations
            # Visualization and output
            show_plot=False,                                                      # display plot
            core_name=CORE_B,                                                    # core name for plot title
            export_csv=False,                                                    # export results to CSV
            mute_mode=True
        )

        # Cell: Multi-Parameter Distribution Analysis

        test_age_constraint_removal = True  # Set to False to disable age constraint removal testing

        # Run all parameter combinations and plot distribution curves together

        # Define all parameter combinations to test
        parameter_combinations = [
            # {'age_consideration': True, 'restricted_age_correlation': True, 'shortest_path_search': False},
            {'age_consideration': True, 'restricted_age_correlation': True, 'shortest_path_search': True},
            # {'age_consideration': True, 'restricted_age_correlation': False, 'shortest_path_search': False},
            # {'age_consideration': True, 'restricted_age_correlation': False, 'shortest_path_search': True},
            # {'age_consideration': False, 'restricted_age_correlation': False, 'shortest_path_search': False},
            {'age_consideration': False, 'restricted_age_correlation': False, 'shortest_path_search': True}
        ]

        # Define all quality indices to process
        target_quality_indices = ['corr_coef', 'norm_dtw', 'perc_diag']

        # Set up output CSV filenames
        if LOG_COLUMNS == ['hiresMS']:
            log_suffix = 'MSonly'
        elif LOG_COLUMNS == ['hiresMS','CT', 'Lumin']:
            log_suffix = 'MSCTLumin'
        else:
            log_suffix = 'unspecified'

        output_csv_filenames = {}
        for quality_index in target_quality_indices:
            if quality_index == 'corr_coef':
                output_csv_filenames[quality_index] = f'outputs/r-values_fit_params_{log_suffix}_{CORE_A}_{CORE_B}.csv'
            else:
                output_csv_filenames[quality_index] = f'outputs/{quality_index}_fit_params_{log_suffix}_{CORE_A}_{CORE_B}.csv'

        # Execute the analysis function
        run_multi_parameter_analysis(
            # Core data inputs
            log_a=log_a, 
            log_b=log_b, 
            md_a=md_a, 
            md_b=md_b,
            all_depths_a_cat1=all_depths_a_cat1,
            all_depths_b_cat1=all_depths_b_cat1,
            pickeddepth_ages_a=pickeddepth_ages_a,
            pickeddepth_ages_b=pickeddepth_ages_b,
            age_data_a=age_data_a,
            age_data_b=age_data_b,
            uncertainty_method=uncertainty_method,
            
            # Analysis parameters
            parameter_combinations=parameter_combinations,
            target_quality_indices=target_quality_indices,
            test_age_constraint_removal=test_age_constraint_removal,
            
            # Core identifiers
            core_a_name=CORE_A,
            core_b_name=CORE_B,
            
            # Output configuration
            output_csv_filenames=output_csv_filenames,
            
            # Optional parameters
            log_columns=LOG_COLUMNS
        )
        
        print(f"Completed processing: {CORE_A} vs {CORE_B}")
        
        # Clean up memory before next iteration
        del log_a, md_a, log_b, md_b
        del pickeddepth_a_csv, pickeddepth_b_csv
        del picked_data_a, picked_data_b
        del all_depths_a_cat1, all_depths_b_cat1
        del consider_adjacent_core, data_columns, age_base_path
        del age_data_a, age_data_b
        del uncertainty_method
        del pickeddepth_ages_a, pickeddepth_ages_b
        del output_csv_filenames
        
        # Force garbage collection
        import gc
        gc.collect()

print(f"\n{'='*80}")
print("All core pair combinations have been processed!")
print(f"{'='*80}")